In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import AlexNet , AlexNet_Weights
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from tqdm import tqdm
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
path_to_data = "/dogsvscats/"

normalizer = transforms.Normalize(
    mean = [0.485, 0.456, 0.406],
    std = [0.229,0.224,0.225]
)

train_transforms = transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.RandomForizontalFlip(),
        transforms.ToTensor(),
        normalizer
    ]
)

dataset = ImageFolder(path_to_data,transform= train_transforms)

train_samples, test_samples = int(0.9*len(dataset)) , len(dataset)- int(0.9*len(dataset))
train_dataset , val_dataset = torch.utils.data.random_split(dataset, lengths = [train_samples,test_samples])



In [ ]:
model = AlexNet()
print(model)


In [ ]:
model.classifier[1]

In [ ]:
model = AlexNet()
model.classifier[6] = nn.Linear(4096,2)
model

In [ ]:
rand_data = torch.rand(16,3,224,224)
model_output = model(rand_data)
model_output.shape

In [ ]:
# Model Parameters
total_parameters = 0
for name , params in model.named_parameters():
    num_params = int(torch.prod(torch.tensor(params.shape)))
    print(f"{name} : {params.shape}, Num Parameters: {num_params}")
    total_parameters += num_params
print("----------------------")
print(f"Total Parameters in AlexNet Model: {total_parameters}")

In [ ]:
# Training from Scratch
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AlexNet()
model.classifier[6] = nn.Linear(4096,2)
model = model.to(device)

num_epochs = 5
optimizer = optim.Adam(params = model.parameters(), lr = 0.00001)
loss_fn = nn.CrossEntropyLoss()
batch_size = 128

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_loader = DataLoader(val_dataset,batch_size = batch_size,shuffle = False)

def train(model, device, num_epochs, optimizer, loss_fn , batch_size, train_loader, val_loader):
    log_training = {
        "epochs": [],
        "training_loss": [],
        "training_accuracy": [],
        "validation_loss": [],
        "validation_accuracy": []
    }

    for epoch in range(1,num_epochs+1):
        print(f"Epoch {epoch}")
        training_losses , training_accuracies = [], []
        validation_losses, validation_accuracies = [], []

        for image,label in tqdm(train_loader):
            image, label = image.to(device), label.to(device)
            optimizer.zero_grad()
            output = model.forward(image)

            loss = loss_fn(label,output)
            training_losses.append(loss.item())

            predictions = torch.argmax(output,axis = 1)
            accuracy = (predictions==label).sum() / len(predictions)
            training_accuracies.append(accuracy.item())

            loss.backward()
            optimizer.step()

        for image,label in tqdm(val_loader):
            image, label = image.to(device) , label.to(device)
            with torch.no_grad():
                output = model.forward(image)

                loss = loss_fn(output,label)
                validation_losses.append(loss.item())

                predictions = torch.argmax(output,axis = 1)
                accuracy = (predictions==label).sum() / len(predictions)
                validation_accuracies.append(accuracy.item())
            
        training_loss_mean , training_accuracy_mean = np.mean(training_losses), np.mean(training_accuracies)
        validation_loss_mean , validation_accuracy_mean = np.mean(validation_losses) , np.mean(validation_accuracies)

        log_training["epochs"].append(epoch)
        log_training["training_loss"].append(training_loss_mean)
        log_training["training_accuracy"].append(training_accuracy_mean)
        log_training["validation_loss"].append(validation_loss_mean)
        log_training["validation_accuracy"].append(validation_accuracy_mean)

        print("Training Loss: ", training_loss_mean)
        print("Training Accuracy: ", training_accuracy_mean)
        print("Validation Loss: ", validation_loss_mean)
        print("Validation Accuracy: ", validation_accuracy_mean)
    
    return log_training , model

random_init_logs, model = train(
    model = model,
    device= device,
    optimizer=optimizer,
    loss_fn= loss_fn,
    batch_size=batch_size,
    train_loader= train_loader,
    val_loader=val_loader
)

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0','alexnet',pretrained = True)
model.classifier[6] = nn.Linear(4096,2)
model = model.to(device)

num_epochs = 2
optimizer = optim.Adam(params = model.parameters(),lr=0.0001)
loss_fn = nn.CrossEntropyLoss()
batch_size = 128

train_loader = DataLoader(train_dataset,batch_size = batch_size,shuffle = True)
val_loader = DataLoader(val_dataset,batch_size = batch_size,shuffle = False)

random_init_logs , model = train(
    model = model,
    device = device,
    epochs = num_epochs,
    optimizer=optimizer,
    loss_fn=loss_fn,
    batch_size=batch_size,
    train_loader=train_loader,
    val_loader=val_loader,
)


In [ ]:
for name , param in model.named_parameters():
    if "bias" in name:
        print(name)
        print(param)
        break

In [ ]:
for name , param in model.named_parameters():
    print(name)

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model.classifier[6] = nn.Linear(4096,2)

for name , param in model.named_parameters():
    if "classifier.6" not in name:
        params.requires_grad_(False)
    
for name , param in model.named_parameters():
    if 'bias' in name:
        print(name)
        print(param)
        break

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model.classifier[6] = nn.Linear(4096, 2)

for name, param in model.named_parameters():
    if "classifier.6" not in name:
        param.requires_grad_(False) 

model = model.to(device)

epochs = 2
optimizer = optim.Adam(params=model.parameters(), lr=0.0001)
loss_fn = nn.CrossEntropyLoss()
batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


random_init_logs, model = train(model=model,
                                device=device,
                                epochs=epochs,
                                optimizer=optimizer,
                                loss_fn=loss_fn,
                                batch_size=batch_size,
                                trainloader=train_loader,
                                valloader=val_loader)